# 0 - Arbitrage Transaction Analysis
This notebook solely analyzes some given arbitrage transactions on Ethereum. It is independent of the notebook series that starts from 1.

See the [log file](arb_tx_data\0xe976f2ebd70135153e8522c3aaa3c9d489717ef621d69b9e00a9f215d0474918\log.txt) for the tx. For analysis purpose, swaps are grouped together if they form a cyclic arbitrage.

In [1]:
# Standard Library
from pprint import pprint

import numpy as np

# Local Folder Library
from pyammanalysis.amm_arb import graph_arbitrage, UniV2Scraper
from pyammanalysis.subgraph import *
from pyammanalysis.util import *

In [2]:
TX_QUERY = """
{
    transaction(id: "0xe976f2ebd70135153e8522c3aaa3c9d489717ef621d69b9e00a9f215d0474918") {
        blockNumber
        timestamp
        gasUsed
        gasPrice
        swaps {
            pool {
                feeTier
            }
            token0 {
                symbol
            }
            token1 {
                symbol
            }
        }
    }
}
"""

In [3]:
arb_tx_result = run_query(UNISWAP_V3_SUBGRAPH_URL, TX_QUERY)["transaction"]
pprint(arb_tx_result)

{'blockNumber': '14931986',
 'gasPrice': '24195653806',
 'gasUsed': '1405319',
 'swaps': [{'pool': {'feeTier': '3000'},
            'token0': {'symbol': 'erowan'},
            'token1': {'symbol': 'USDT'}},
           {'pool': {'feeTier': '10000'},
            'token0': {'symbol': 'WETH'},
            'token1': {'symbol': 'MTO'}},
           {'pool': {'feeTier': '10000'},
            'token0': {'symbol': 'RSR'},
            'token1': {'symbol': 'WETH'}}],
 'timestamp': '1654768688'}


In [4]:
block_number = int(arb_tx_result["blockNumber"])
block_number

14931986

## Group 5
Cyclic arbitrage of 3 pools on Uniswap V2:
```
// group 5: ETH -> ALBT -> 5GS -> ETH, Uniswap V2 => earns ETH
Swap 0.078344525103934434 Ether         For 1,754.673155155967912284 ALBT       On Uniswap V2
Swap 1,754.673155155967912284 ALBT      For 6,645.694535765537281985 GS         On Uniswap V2
Swap 6,645.694535765537281985 GS        For 0.082361461143867988 Ether          On Uniswap V2
```

In [5]:
v2_scraper = UniV2Scraper(block_number)
result = v2_scraper.tx_query()

In [6]:
# note: only prices of pairs in the cyclic arbitrage are filled in
v2_adj_matrix = v2_scraper.create_adj_matrix(
    "data/univ2_adj_matrix.csv", pairs=result["pairs"], tokens=result["tokenSymbols"]
)
v2_adj_matrix.astype(np.float64)  # for display purpose

C:\Users\user\Documents\GitHub\eth-uniswap-prelim-analysis\pyammanalysis\amm_arb\uniswapv2_scraper.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[token1][token0] = pair["token0Price"]
C:\Users\user\Documents\GitHub\eth-uniswap-prelim-analysis\pyammanalysis\amm_arb\uniswapv2_scraper.py:162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[token0][token1] = pair["token1Price"]


,LSS,LYM,ALBT,ALN,RSR,USDT,LMT,MTO,WETH,JRT,STND,GS
LSS,NaN,NaN,NaN,NaN,NaN,8.123955,NaN,NaN,8972.587921,NaN,NaN,NaN
LYM,NaN,NaN,NaN,NaN,NaN,NaN,0.774994,NaN,NaN,NaN,NaN,NaN
ALBT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21851.332174,NaN,NaN,0.265061
ALN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83861.943900,NaN,NaN,NaN
RSR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123136.884796,NaN,NaN,NaN
USDT,0.123093,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LMT,NaN,1.290333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MTO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26358.182387,NaN,NaN,NaN
WETH,0.000111,NaN,0.000046,0.000012,0.000008,NaN,NaN,0.000038,NaN,0.000014,0.000044,0.000013
JRT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73422.541357,NaN,NaN,NaN


In [7]:
graph_arbitrage.find_arbitrage(filename="data/univ2_adj_matrix.csv", find_all=True)

ARBITRAGE FOUND

Path: ['WETH', 'ALBT', 'GS', 'WETH']
6.2%

Path: ['GS', 'ALBT', 'GS']
2.2e-14%



[['WETH', 'ALBT', 'GS', 'WETH'], ['GS', 'ALBT', 'GS']]